Text Simplification

In [ ]:
#installing all the dependencies
!git clone https://github.com/facebookresearch/muss.git   
!cd muss
!pip install -e .
!python -m spacy download en_core_web_md fr_core_news_md es_core_news_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/muss
  Cloning https://github.com/feralvam/easse.git to /tmp/pip-install-5i_37kh0/easse_dee4896aeb0f4cae9b03e337db20ce61
  Running command git clone -q https://github.com/feralvam/easse.git /tmp/pip-install-5i_37kh0/easse_dee4896aeb0f4cae9b03e337db20ce61
  Cloning https://github.com/kpu/kenlm.git to /tmp/pip-install-5i_37kh0/kenlm_c4853cc049574e5cbdc06edcc131135b
  Running command git clone -q https://github.com/kpu/kenlm.git /tmp/pip-install-5i_37kh0/kenlm_c4853cc049574e5cbdc06edcc131135b
  Cloning https://github.com/facebookresearch/text-simplification-evaluation.git (to revision main) to /tmp/pip-install-5i_37kh0/tseval_c3bbbe49f8be4726b4d53e2b1125cc3f
  Running command git clone -q https://github.com/facebookresearch/text-simplification-evaluation.git /tmp/pip-install-5i_37kh0/tseval_c3bbbe49f8be4726b4d53e2b1125cc3f
  Attempting uninstall: muss
    Found exis

In [ ]:
!python scripts/simplify.py scripts/examples.en --model-name muss_en_wikilarge_mined

tcmalloc: large alloc 1625161728 bytes == 0x74328000 @  0x7f19c5ed4b6b 0x7f19c5ef4379 0x7f1944576d57 0x7f1944564bc3 0x7f1996334811 0x5d7f65 0x56376d 0x55e858 0x5d7cf1 0x4eb089 0x5d86fe 0x654d9d 0x58d798 0x4f7763 0x49ca7c 0x55e858 0x5d7cf1 0x5d77c6 0x561051 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x55e571 0x5d7cf1 0x49ec69 0x55e571 0x5d7cf1 0x49ced5
tcmalloc: large alloc 1625161728 bytes == 0xd5108000 @  0x7f19c5ed4b6b 0x7f19c5ef4379 0x7f1944576d57 0x7f1944564bc3 0x7f1996334811 0x5d7f65 0x56376d 0x55e858 0x5d7cf1 0x4eb089 0x5d86fe 0x654d9d 0x58d798 0x4f7763 0x49ca7c 0x55e858 0x5d7cf1 0x5d77c6 0x561051 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x55e571 0x5d7cf1 0x49ec69 0x55e571 0x5d7cf1 0x49ced5
  0% 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This result

Abstractive summarization module

In [ ]:
from transformers import BartTokenizer, PegasusTokenizer
from transformers import BartForConditionalGeneration, PegasusForConditionalGeneration

IS_CNNDM = True # whether to use CNNDM dataset or XSum dataset
LOWER = False

ARTICLE_TO_SUMMARIZE = "When you have climbed to the top, the mountain, whose slopes are covered with forest mangoes, glowing with ripe fruit, takes on the appearance of a breast of the earth, dark at the centre, the rest pale, worthy to be beheld by a divine couple. "

# Load our model checkpoints
if IS_CNNDM:
    model = BartForConditionalGeneration.from_pretrained('Yale-LILY/brio-cnndm-uncased')
    tokenizer = BartTokenizer.from_pretrained('Yale-LILY/brio-cnndm-uncased')
else:
    model = PegasusForConditionalGeneration.from_pretrained('Yale-LILY/brio-xsum-cased')
    tokenizer = PegasusTokenizer.from_pretrained('Yale-LILY/brio-xsum-cased')

max_length = 1024 if IS_CNNDM else 512
# generation example
if LOWER:
    article = ARTICLE_TO_SUMMARIZE.lower()
else:
    article = ARTICLE_TO_SUMMARIZE
inputs = tokenizer([article], max_length=max_length, return_tensors="pt", truncation=True)
# Generate Summary
summary_ids = model.generate(inputs["input_ids"])
print("The result from abstractive summarization is")
summary=tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(summary)

The result from abstractive summarization is
The mountain's slopes are covered with forest mangoes, glowing with ripe fruit. It takes on the appearance of a breast of the earth, dark at the centre and pale at the top.    the mountain is covered in mangoes and mangoes.  


Sentence splitting.

In [ ]:
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

from typing import List
import numpy as np
import nltk
from itertools import chain
nltk.download('punkt')
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, LSTM, Concatenate, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import logging
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/MyDrive

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
class SentenceSplitter():
    def __init__(self,
            sentence_length,
            lstm_layers_size,
            embedding_layer_size,
            dropout_rate,
            activation,
            epochs
    ):
        super().__init__()
        self.sentence_length = sentence_length
        self.lstm_layers_size = lstm_layers_size
        self.embedding_layer_size = embedding_layer_size
        self.dropout_rate = dropout_rate
        self.activation = activation
        self.epochs = epochs

    def sent2seq(self, sentence, padding_location="pre"):
        sequences = self.tokenizer.texts_to_sequences([sentence])
        return pad_sequences(sequences, maxlen=self.sentence_length, dtype='int32', padding=padding_location,
            truncating=padding_location, value=0.0).squeeze()

    def create_dataset(self, texts: List[str], min_sentence_length_for_splitting=10):
        '''
        Create positives and negatives pairs of sentences.
        Positive pair is a pair of sentences that should be splitted, using nltk.sent_tokenize (without \n).
        Negative pair is a sentence that we split in the middle
        '''
        positives_pairs = []
        negative_pairs = []
        for text in texts:
            sentences = nltk.sent_tokenize(text)
            if len(sentences)>1:
                sentences = [nltk.tokenize.word_tokenize(sentence.replace('.', '')) for sentence in sentences]
                sentences_pairs = list(zip(sentences[:-1], sentences[1:]))
                positives_pairs.append(sentences_pairs)

                for negative_sample in sentences:  # split sentence in the middle
                    if len(negative_sample) > min_sentence_length_for_splitting:
                        first_sent, second_sent = negative_sample[:len(negative_sample) // 2], negative_sample[
                                                                                                len(negative_sample) // 2:]
                        negative_pairs.append([(first_sent, second_sent)])
        
        positives_pairs = list(chain.from_iterable(positives_pairs))
        negative_pairs = list(chain.from_iterable(negative_pairs))
        
        return positives_pairs, negative_pairs

    def get_cases(self):
        myfile = open("wikisent2.txt", "rt")
        contents = myfile.read() 
        myfile.close()
        return contents.split('\n')[::8]

    def read_dataset(self, texts, positives_pairs, negative_pairs):
        self.tokenizer = Tokenizer()
        corpus = [[nltk.tokenize.word_tokenize(sentence) for sentence in nltk.sent_tokenize(text)] for text in texts]
        corpus =list(chain.from_iterable(corpus)) 
        self.tokenizer.fit_on_texts(corpus)
        positives_pairs = np.array([[self.sent2seq(sent1, padding_location="pre"),
                                        self.sent2seq(sent2, padding_location="post")] for sent1, sent2 in
                                    positives_pairs])
        negative_pairs = np.array([[self.sent2seq(sent1, padding_location="pre"),
                                    self.sent2seq(sent2, padding_location="post")] for sent1, sent2 in negative_pairs])
        X = np.concatenate((positives_pairs, negative_pairs))
        y = np.array([1] * len(positives_pairs) + [0] * len(negative_pairs))
        return X, y

    def fit(self, X, y):
        '''
        X: sentences pairs
        y: 1- represent splitted sentences, 0- represent merged sentences
        '''
        weights = y * (len(y) - sum(y)) / sum(y)  # positive weight
        weights[weights == 0] = 1  # negative weight
        logging.info(f"Number of positives: {len(y[y == 1])}, Number of negatives: {len(y[y == 0])}")
        vocab_size = len(self.tokenizer.word_index) + 1

        # first sentence forward
        forward_model_input = Input(shape=(X.shape[-1],))
        forward_model = Embedding(vocab_size, self.embedding_layer_size)(forward_model_input)
        forward_model = self.add_lstm_layers(forward_model, go_backwards=False)

        # second sentence backward
        backward_model_input = Input(shape=(X.shape[-1],))
        backward_model = Embedding(vocab_size, self.embedding_layer_size)(backward_model_input)
        backward_model = self.add_lstm_layers(backward_model, go_backwards=True)

        model_concatenated = Concatenate()([forward_model, backward_model])
        model_concatenated = Dense(self.lstm_layers_size[-1] * 2, activation=self.activation)(model_concatenated)

        model_concatenated = Dense(1, activation='sigmoid')(model_concatenated)
        self.model = Model(inputs=[forward_model_input, backward_model_input], outputs=model_concatenated)

        logging.info(f"Dataset shape: {X.shape}")
        self.model.summary(print_fn=logging.getLogger(__name__).info)

        self.model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        self.model.fit([X[:, 0, :], X[:, 1, :]], y, batch_size=256, epochs=self.epochs, shuffle=True,
            sample_weight=weights)
    def add_lstm_layers(self, model, go_backwards=False):
        for i, layer_dim in enumerate(self.lstm_layers_size):
            if i == len(self.lstm_layers_size) - 1:  # last layer
                model = LSTM(layer_dim, return_sequences=False, dropout=self.dropout_rate,
                    go_backwards=go_backwards)(model)
            else:
                model = LSTM(layer_dim, return_sequences=True, dropout=self.dropout_rate,
                    go_backwards=go_backwards)(model)
        return model
    
    def transform(self, sentences):
        if len(sentences) == 0:
            return []
        splitted_sentences = [sentences[0]]
        for sentence in sentences[1:]:
            previous_sentence = splitted_sentences.pop()
            pair_sentences = np.array([self.sent2seq(nltk.tokenize.word_tokenize(previous_sentence), padding_location="pre"),
                                       self.sent2seq(nltk.tokenize.word_tokenize(sentence), padding_location="post")])

            prob_split= self.model.predict([pair_sentences[0, :].reshape(1, self.sentence_length),
                                                 pair_sentences[1, :].reshape(1, self.sentence_length)])[:, 0][0]
            if prob_split > 0.5:
                splitted_sentences += [previous_sentence, sentence]
            else: #merge
                splitted_sentences += [(previous_sentence + ' ' + sentence)]

        return splitted_sentences
    

In [ ]:
model = SentenceSplitter(sentence_length=80,
            lstm_layers_size= (64, 32),
            embedding_layer_size = 100,
            dropout_rate= 0.3,
            activation= 'softmax',
            epochs=10)
results = SentenceSplitter.final_resut(summary)
print(results)

["The mountain's slopes are covered with forest mangoes, glowing with ripe fruit", 'it takes on the appearance of a breast of the earth, dark at the centre and pale at the top', 'The mountain is covered in mangoes and mangoes in the summer.']


Keyword extractor

In [ ]:
from wordwise import Extractor
extractor = Extractor()
i="https://docs.google.com/presentation/d/1gd08BOQzG4miaxPJZ_iZ7xSFPJ8IEllS/edit#slide=id.p1"
length = 3
if(len(i)>=10):
  length=4
if(len(i)>20):
    length=5
keywords = extractor.generate(i,length)
print(keywords)